In [79]:
import time
import pandas as pd
import numpy as np
import itertools

start = time.time()

full_tutors = pd.read_csv("tutors.csv")
full_classes = pd.read_csv("classes.csv")
full_students = pd.read_csv("students.csv")

In [80]:
# minor preprocessing of original dataframes for better organization later
full_tutors = full_tutors.rename(columns={'Name' : 'Tutor'})
full_classes = full_classes.rename(columns={'Class #1 Name (ex. 5th Grade Strings, 7th Grade Concert Band, etc)': 'Class #1 Name', 'Class #1 Classroom Number (If you do not have a specific room number, please provide a description of where the class is held)': 'Classroom Number #1'})
full_classes = full_classes.rename(columns={'Class #2 Name (ex. 5th Grade Strings, 7th Grade Concert Band, etc)': 'Class #2 Name', 'Class #2 Classroom Number (If you do not have a specific room number, please provide a description of where the class is held)': 'Classroom Number #2'})
full_classes = full_classes.rename(columns={'Class #3 Name (ex. 5th Grade Strings, 7th Grade Concert Band, etc)': 'Class #3 Name', 'Class #3 Classroom Number (If you do not have a specific room number, please provide a description of where the class is held)': 'Classroom Number #3'})
full_students = full_students.rename(columns={'Student Name - Last, First': 'Student'})
# full_tutors and full_students dataframes will not be mutated after this

In [81]:
# split students into the 4 priorities 
application = 'Previous application with TMC'
freeLunch = 'Does the student qualify for the free or reduced lunch program?'
prevTutored = 'Yes, my student has previously applied and received lessons with TMC'
prevApplied = 'Yes, my student has applied for lessons with TMC before but was not placed for lessons'
newStudent = 'No, this is my student\'s first time applying for lessons with TMC'

students1 = full_students.loc[(full_students[application] == prevTutored)]
students2 = full_students.loc[(full_students[application] == prevApplied) & (full_students[freeLunch] == 'Yes')]
students3 = full_students.loc[(full_students[application] == newStudent) & (full_students[freeLunch] == 'Yes')]
students4 = full_students.loc[(full_students[application] == prevApplied) & (full_students[freeLunch] == 'No')]
students5 = full_students.loc[(full_students[application] == newStudent) & (full_students[freeLunch] == 'No')]

In [82]:
#split classes into separate rows
classes1 = full_classes[['Email address','Name', 'School', 'Class #1 Name', 'Class #1 Grade Level', 'Class #1 Instruments (check all that apply)','Class #1 Timings [Monday]', 'Class #1 Timings [Tuesday]', 'Class #1 Timings [Wednesday]', 'Class #1 Timings [Thursday]', 'Class #1 Timings [Friday]', 'Classroom Number #1', 'How specifically would you like the tutors to help you? (ex. one-on-one, sectionals, etc.)', 'Any other requests or concerns that you would like TMC to know?']]
classes2 = full_classes[['Email address','Name', 'School', 'Class #2 Name', 'Class #2 Grade Level', 'Class #2 Instruments (check all that apply)','Class #2 Timings [Monday]', 'Class #2 Timings [Tuesday]', 'Class #2 Timings [Wednesday]', 'Class #2 Timings [Thursday]', 'Class #2 Timings [Friday]', 'Classroom Number #2', 'How specifically would you like the tutors to help you? (ex. one-on-one, sectionals, etc.)', 'Any other requests or concerns that you would like TMC to know?']]
classes3 = full_classes[['Email address','Name', 'School', 'Class #3 Name', 'Class #3 Grade Level', 'Class #3 Instruments (check all that apply)','Class #3 Timings [Monday]', 'Class #3 Timings [Tuesday]', 'Class #3 Timings [Wednesday]', 'Class #3 Timings [Thursday]', 'Class #3 Timings [Friday]', 'Classroom Number #3', 'How specifically would you like the tutors to help you? (ex. one-on-one, sectionals, etc.)', 'Any other requests or concerns that you would like TMC to know?']]
new_cols = {x: y for x, y in zip(classes2.columns, classes1.columns)}
classes1 = classes1.append(classes2.rename(columns=new_cols), ignore_index = True)
new_cols = {x: y for x, y in zip(classes3.columns, classes1.columns)}
classes1 = classes1.append(classes3.rename(columns=new_cols), ignore_index = True)
classes1 = classes1[classes1['Class #1 Name'].isna() == False].reset_index(drop=True)
full_classes = classes1.rename({'Class #1 Name': 'Class Name', 'Class #1 Grade Level': 'Grade Level','Class #1 Instruments (check all that apply)': 'Instruments', 'Class #1 Timings [Monday]': 'Monday', 'Class #1 Timings [Tuesday]': 'Tuesday', 'Class #1 Timings [Wednesday]': 'Wednesday', 'Class #1 Timings [Thursday]': 'Thursday', 'Class #1 Timings [Friday]':'Friday', 'Classroom Number #1': 'Classroom Number'},axis=1)
full_classes

,Email address,Name,School,Class Name,Grade Level,Instruments,Monday,Tuesday,Wednesday,Thursday,Friday,Classroom Number,"How specifically would you like the tutors to help you? (ex. one-on-one, sectionals, etc.)",Any other requests or concerns that you would like TMC to know?
0,hsdfhj@ngsd,hjdhfsj,sdfjh,sjfh,3-5,"Violin, Viola, Cello, Flute, Clarinet, Saxopho...","10AM, 11AM","10AM, 11AM",NaN,"10AM, 11AM",NaN,231,NaN,NaN
1,vaibg@berkeley.edu,Vaibhav,Berkeley,Percussion,3-5,"Percussion, Piano",NaN,NaN,11AM,NaN,NaN,Dwinelle 89,One-on-one help,Backdoor
2,asdf@gmail.com,asdf fdsa,asdf high,8 gr bamd,6-8,"Violin, Viola, Cello, Bass, Piano",NaN,"1PM, 2PM",NaN,"1PM, 2PM",NaN,313,wee,no
3,ahmed.baqai@gmail.com,Jake Pail,Cupertino,5th,9-12,"Tuba, Piano, Voice","8AM, After 4PM","9AM, 3PM","10AM, 11AM, 1PM, 2PM","11AM, 1PM",12PM,23,one-on-one,NaN
4,hepsdafhs,kjadfhdasf,skdfhwkjh,sdafads,K-2,Voice,2PM,NaN,2PM,NaN,2PM,123,NaN,NaN
5,wqeqw@gmail.com,aawer,qqwer high,aaaa 2,K-2,"Flute, Clarinet, Saxophone, Oboe, Trombone, Tr...",3PM,NaN,3PM,NaN,3PM,1515,1515,no
6,kashfdsfh,kjsadhfak,kajsdhfds,askdfhskj,9-12,Voice,1PM,1PM,NaN,1PM,1PM,asjdfh,NaN,NaN
7,asf@gmail.com,Johnny Test,Homestead High,String Orchestra,9-12,"Violin, Bass, Flute, Trumpet, Percussion, Voice","8AM, 1PM","9AM, 2PM","11AM, 1PM","1PM, 3PM",1PM,20,No Sir,No Sir
8,johnson@gmail.com,Jing Howard,Monta Vista,Denvor percussion,3-5,"Tuba, Percussion","8AM, 9AM",10AM,11AM,"9AM, 12PM","9AM, 11AM",54,One-on-one,None
9,vaibg@berkeley.edu,Ahmed,Cindy,Danie,9-12,"Violin, Viola, Cello, Bass, Clarinet",1PM,11AM,NaN,10AM,"2PM, 3PM",Now,Easy,No


In [83]:
# more useful arrays
student = ['Student']
choices = ['Instrument (First Choice)', 'Instrument (Second Choice)', 'Instrument (Third Choice)']
grade = ['Grade Level']

days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
times = ['Before 11am', '11am-1pm', '1pm-3pm', '3pm-5pm', "After 5pm"]
daystimes = [day + ' ' + time for day in days for time in times]

class_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
class_times = ['8AM', '9AM', '10AM', '11AM', '12PM', '1PM', '2PM', '3PM', 'After 4PM']
class_daystimes = [day + ' ' + time for day in class_days for time in class_times]

rangeToList = {'Before 11am': '8AM, 9AM, 10AM', '11am-1pm':'11AM, 12PM', '1pm-3pm':'1PM, 2PM', '3pm-5pm':'3PM, After 4PM', "After 5pm":'After 4PM'}
grades = ['K-2', '3-5', '6-8', '9-12']
gradesToList = {'K-2': 'K,1,2', '3-5': '3,4,5', '6-8': '6,7,8', '9-12':'9,10,11,12'}

In [84]:
"""
Function to preprocess the given dataframe.
Arguments: data - raw dataframe (read from tutors.csv or students.csv)
           status - 'students' or 'tutors'
Returns: new dataframe with only the necessary columns for matching
"""
def preprocess(data, status):
    # filter out irrelevant columns, remove/replace invalid data entries
    data = data.replace(np.nan, '')
    if status == 'students':
        data = data[student + choices + days + grade]
        # consolidate all instruments into one column, in the order of preference
        data['Instrument'] = data[choices].agg(', '.join, axis=1)
        data = data.drop(choices, 1)
        data = data.rename(columns={'Grade Level': 'Grade'})
        
    elif status == 'tutors':
        data = data[['Tutor', 'Which instrument(s) would you like to teach?'] + days + ['Grade levels you prefer to work with:']]
        data = data.rename(columns={'Which instrument(s) would you like to teach?': 'Instrument'})
        data = data.rename(columns={'Grade levels you prefer to work with:': 'Grade'})
        index = 0
        for g in grades:
            data['Grade'] = data['Grade'].str.replace(g, gradesToList[g])       
        
    # standardize instrument name to be lowercase
    data['Instrument'] = data['Instrument'].str.lower()
    
    # extract day/time availabilities into separate columns
    index = 0
    for i in range(len(days)):
        for j in range(len(times)):
            d = days[i]
            t = times[j]
            data[daystimes[index]] = data[d].str.contains(t)
            index += 1
    
    # keep only day, time combination columns in dataframe
    data = data.drop(days, 1)
    
    # add column for number of availabilities
    temp = data[daystimes]
    temp['Count'] = temp.sum(1)
    data['Count'] = temp['Count']
    data = data.sort_values(by='Count')
    
    return data

In [85]:
"""
Function to preprocess the given dataframe.
Arguments: data - raw dataframe (read from tutors.csv or students.csv)
           status - 'students' or 'tutors'
Returns: new dataframe with only the necessary columns for matching
"""
def preprocess_for_classes(data):
    # filter out irrelevant columns, remove/replace invalid data entries
    data = data.replace(np.nan, '')

    data = data[['Tutor', 'Which instrument(s) would you like to teach?'] + days + ['Grade levels you prefer to work with:']]
    data = data.rename(columns={'Which instrument(s) would you like to teach?': 'Instrument'})
    data = data.rename(columns={'Grade levels you prefer to work with:': 'Grade'})
    
    
    data['Monday'] = data['Monday'].str.replace('Before 11am', '8AM, 9AM, 10AM')
    
    # standardize instrument name to be lowercase
    data['Instrument'] = data['Instrument'].str.lower()
    
    index = 0
    for i in range(len(days)):
        for j in range(len(times)):
            d = days[i]
            t = times[j]
            data[d] = data[d].str.replace(t, rangeToList[t])
            index += 1
    data = data.drop(['Saturday', 'Sunday'],1)
    data = extract_times(data)

    return data

def extract_times(data):
    # extract day/time availabilities into separate columns
    data = data.replace(np.nan, '')
    index = 0
    for i in range(len(class_days)):
        for j in range(len(class_times)):
            d = class_days[i]
            t = class_times[j]
            data[class_daystimes[index]] = data[d].str.contains(t)
            index += 1
    
    # keep only day, time combination columns in dataframe
    data = data.drop(class_days, 1)
    
    # add column for number of availabilities
    temp = data[class_daystimes]
    temp['Count'] = temp.sum(1)
    data['Count'] = temp['Count']
    data = data.sort_values(by='Count')
    data = data.drop('Count', axis=1)
    return data


In [86]:
# create dataframes to use for matching
pd.set_option('mode.chained_assignment', None)
tutors = preprocess(full_tutors, 'tutors')
# students = preprocess(full_students, 'students')

students1 = preprocess(students1, 'students')
students2 = preprocess(students2, 'students')
students3 = preprocess(students3, 'students')
students4 = preprocess(students4, 'students')
students5 = preprocess(students5, 'students')

# all students, in order of priority level and least to most available
students = pd.concat([students1, students2, students3, students4, students5])

#class matching
tutors_for_classes = preprocess_for_classes(full_tutors)
classes = extract_times(full_classes)

In [87]:
# copy to mutate in function; tutors will be removed once they are matched
tutors2 = tutors.copy()
t_without_count = tutors2.drop(columns='Count')

In [105]:
"""
Takes in a student in the form of a tuple with (length, Series) -- values from a df.iterrows() iterable.
Returns a list of matched tutor, instrument, and time slots, or a list of Nones if no match can be created.
Matches are considered valid if there is an instrument overlap between student and tutor, and at least one time availability overlap.
"""
def match(student):
    student_info = student[1]
    # remove empty strings from student's instruments choice (i.e. if student has less than 3 preferences)
    student_instruments = list(filter(None, student_info['Instrument'].split(', ')))
    student_grade = student_info['Grade']
        
    # iterate through student's instruments, beginning with first choice
    for instrument in student_instruments:
        # iterate through all remaining tutors, searching for a match with specific instrument choice
        tutors_iter = t_without_count.iterrows()
        for tutor in tutors_iter:
            tutor_info = tutor[1]
            tutor_instruments = tutor_info['Instrument']
            tutor_grade = list(filter(None, tutor_info['Grade'].split(', ')))
            # merge into one Series with values as lists of [tutor value, student value]
            combined_info = tutor_info.combine(student_info, lambda x, y: [x, y])
            # times where both tutor and student are available
            times = [t for t in combined_info.index if combined_info[t] == [True, True]]
            
            #brute force grade preference
            if instrument in tutor_instruments and len(times) > 0 and student_grade in tutor_grade:
                return [tutor_info[0], instrument, times]
        
        for tutor in tutors_iter:
            tutor_info = tutor[1]
            tutor_instruments = tutor_info['Instrument']
            tutor_grade = list(filter(None, tutor_info['Grade'].split(', ')))
            # merge into one Series with values as lists of [tutor value, student value]
            combined_info = tutor_info.combine(student_info, lambda x, y: [x, y])
            # times where both tutor and student are available
            times = [t for t in combined_info.index if combined_info[t] == [True, True]]
            
            # if instrument matches and there is at least one shared time availability
            if instrument in tutor_instruments and len(times) > 0:
                return [tutor_info[0], instrument, times]
    
    # no match found for any of the three instrument choices
    return [None, None, None]

In [100]:
"""
Takes in a class in the form of a tuple with (length, Series) -- values from a df.iterrows() iterable.
Returns a list of matched tutor, instrument, and time slots, or a list of Nones if no match can be created.
Matches are considered valid if there is an instrument overlap between student and tutor, and at least one time availability overlap.
"""
def match_classes(classes):
    class_info = classes[1]
    # remove empty strings from student's instruments choice (i.e. if student has less than 3 preferences)
    class_instruments = list(filter(None, class_info['Instruments'].split(', ')))
    class_grade = class_info['Grade Level']
        
    # iterate through student's instruments, beginning with first choice
    for instrument in class_instruments:
        # iterate through all remaining tutors, searching for a match with specific instrument choice
        tutors_iter = tutors_for_classes.iterrows()
        for tutor in tutors_iter:
            tutor_info = tutor[1]
            tutor_instruments = tutor_info['Instrument']
            tutor_grade = list(filter(None, tutor_info['Grade'].split(', ')))
            # merge into one Series with values as lists of [tutor value, student value]
            combined_info = tutor_info.combine(class_info, lambda x, y: [x, y])
            # times where both tutor and student are available
            times = [t for t in combined_info.index if combined_info[t] == [True, True]]
            
            #brute force grade preference
            if instrument in tutor_instruments and len(times) > 0 and class_grade in tutor_grade:
                return [tutor_info[0], instrument, times]
        
        for tutor in tutors_iter:
            tutor_info = tutor[1]
            tutor_instruments = tutor_info['Instrument']
            tutor_grade = list(filter(None, tutor_info['Grade'].split(', ')))
            # merge into one Series with values as lists of [tutor value, student value]
            combined_info = tutor_info.combine(class_info, lambda x, y: [x, y])
            # times where both tutor and student are available
            times = [t for t in combined_info.index if combined_info[t] == [True, True]]
            
            # if instrument matches and there is at least one shared time availability
            if instrument in tutor_instruments and len(times) > 0:
                return [tutor_info[0], instrument, times]
    
    # no match found for any of the three instrument choices
    return [None, None, None]

In [101]:
# create the iterator from rows of students
s_without_count = students.drop(columns='Count')
students_iter = s_without_count.iterrows()

In [102]:
# create the dataframe to store matches once made
matches = pd.DataFrame(columns=['Student', 'Tutor', 'Instrument', 'Time(s)'])
class_matches = pd.DataFrame(columns=['Class', 'Tutor', 'Instrument', 'Time(s)'])

In [103]:
# iterate through students; use function defined above to get a match
for c in classes.iterrows():
    m = match_classes(c)
    # add to matches dataframe
    class_matches.loc[len(class_matches)] = [c[1][0]] + m
    # remove tutor if successfully matched
    if m[0] is not None:
        tutors2 = tutors2.drop(tutors2[tutors2['Tutor'] == m[0]].index)
class_matches

,Class,Tutor,Instrument,Time(s)
0,vaibg@berkeley.edu,None,None,None
1,hepsdafhs,None,None,None
2,wqeqw@gmail.com,None,None,None
3,kashfdsfh,None,None,None
4,asdf@gmail.com,None,None,None
5,kashfdsfh,None,None,None
6,vaibg@berkeley.edu,None,None,None
7,vaibg@berkeley.edu,None,None,None
8,hepsdafhs,None,None,None
9,hsdfhj@ngsd,None,None,None


In [106]:
# iterate through students; use function defined above to get a match
for s in students_iter:
    m = match(s)
    # add to matches dataframe
    matches.loc[len(matches)] = [s[1][0]] + m
    # remove tutor if successfully matched
    if m[0] is not None:
        tutors2 = tutors2.drop(tutors2[tutors2['Tutor'] == m[0]].index)



In [94]:
# matched students
matched = matches[matches['Tutor'].notna()]

In [95]:
# unmatched students
no_match = matches[matches['Tutor'].isna()]

In [96]:
# unmatched tutors
tutors2[['Tutor', 'Instrument', 'Count']]

,Tutor,Instrument,Count
14,help,guitar,4
11,askfhalkjh,"clarinet, piano",7
2,Jose,piano,8
4,Lizzy,"violin, bass, piano",8
3,zz@gmail.com,"piano, guitar",9
12,gah,viola,9
1,Rohan,"flute, piano, guitar",10
10,Bob the Builder,"bass, clarinet",10
15,sad,guitar,10
13,wat,guitar,11


In [97]:
# matches with relevant information (add any columns needed)
matched = matched.merge(full_students[['Student', 'Email Address']], on='Student', how='left')
matched = matched.merge(full_tutors[['Tutor', 'Email']], on='Tutor', how='left')
matched = matched.rename(columns={'Email Address' : 'Student Email', 'Email' : 'Tutor Email'})
matched

,Instrument,Time(s),Student,Student Email,Tutor,Tutor Email


In [98]:
# students with no matches with relevant information (add any columns needed)
no_match = no_match.merge(full_students[['Student', 'Email Address']], on='Student', how='left')
no_match = no_match.rename(columns={'Email Address' : 'Student Email'}).drop(['Tutor', 'Instrument', 'Time(s)'], 1)
no_match

,Student,Student Email
0,"Mosey, Lil",lm@gmail.com
1,"Bob, Alice",123movies@gmail.com
2,"berkelee, berk",blahblah@berkeley.edu
3,"quarantine, help",coron@virus.org
4,"Lee, Ender",enderlee2006@yahoo.com
5,"Pitt, Bradasdf",ahmed.baqai@gmail.com
6,"Glover, Donald",gg@gmail.com
7,"Pitt, Brad",ahmed.baqai@gmail.com
8,"Stalin, Joseph",jstalin@gmail.com
9,"cjh, asdkfjh",prims@aol.com


In [63]:
# time for computation (in seconds):
time.time() - start

428.39906883239746